In [ ]:
# Import libraries
import os
from keras.models import Sequential
from keras.layers import Dense, LSTM, Dropout, SimpleRNN
from keras import backend as K
from keras import regularizers

import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

In [ ]:
# Set GPU device
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"]=""

In [ ]:
## fix random seed for reproducibility
#numpy.random.seed(7)

In [ ]:
# Variable definition
data_folder = 'Data/' # Folder containing the dataset
n_days = 1640

In [ ]:
# Preallocate the array
dataset = np.empty((n_days, 0))

# Create list to save assets names
assets = []
for f in sorted(os.listdir(data_folder)):

    # Save assets names
    assets.append(f.replace('.csv', ''))

    # Load data
    asset = pd.read_csv(data_folder + f, sep=',', usecols=[2, 3], engine='python')
    asset = asset.values[:n_days]
 
    # Ensure all data is float
    asset = asset.astype('float32')
    dataset = np.append(dataset, asset, axis=1)

In [ ]:
# Visualize returns and volatility of the first asset
i = 0
plt.plot(dataset[:, 0], label='returns')
plt.plot(dataset[:, 1], label='volatility')
plt.legend()
plt.title(assets[0])
plt.xlabel('Time (days)')
plt.show()

In [ ]:
# Normalize data
factor = 2

# Calculate second raw moment
M2 = np.mean(dataset ** 2, axis=0) ** (1/2)

# Apply scaling
dataset_norm = (1/factor) * (dataset / M2)

In [ ]:
def create_dataset(dataset, look_back=1):
    """
    Function to convert series from dataset to supervised learning problem
    """
    data_x, data_y = [], []

    for i in range(len(dataset) - look_back):

        # Create sequence of length equal to look_back
        x = dataset[i:(i + look_back), :]
        data_x.append(x)

        # Take just the volatility for the target
        data_y.append(dataset[i + look_back, 1::2])

    return np.array(data_x), np.array(data_y)

In [ ]:
# Convert series to supervised learning problem
look_back = 20
X, y = create_dataset(dataset_norm, look_back)

# Declare variables
n_features = dataset.shape[1]
n_assets = y.shape[1]

In [ ]:
# Split dataset
training_days = 300
X_train, X_test = X[:training_days], X[training_days:]
y_train, y_test = y[:training_days], y[training_days:]

# Prepare the 3D input vector for the LSTM
X_train = np.reshape(X_train, (-1, look_back, n_features))
X_test = np.reshape(X_test, (-1, look_back, n_features))

In [ ]:
batch_size = 1

# Create the model
model = Sequential()
model.add(LSTM(58,
               input_shape=(look_back, n_features),
               batch_size=batch_size,
               stateful=True,
               activity_regularizer=regularizers.l1_l2(),
               recurrent_regularizer=regularizers.l1_l2()))
model.add(Dropout(0.2))
model.add(Dense(n_assets, activation='sigmoid'))          

In [ ]:
# Compile the LSTM model
model.compile(loss='mse', optimizer='rmsprop')

In [ ]:
num_epochs = 300

# Fit the model
for i in range(num_epochs):

    model.fit(X_train,
              Y_train,
              batch_size=batch_size,
              shuffle=False,
              epochs=1,
              verbose=0) # Verbosity mode 0: silent

    model.reset_states()

In [ ]:
# Make a prediction (rolling test window)
y_pred = np.empty((0, n_assets))

for i in range(len(X_test)):
    X_i = X_test[i].reshape(1, look_back, n_features)
    predicted_output = model.predict(X_i, batch_size=batch_size)

    # Reshape prediction to save into array
    predicted_output = predicted_output.reshape(1,n_assets)
    y_pred = np.append(y_pred, predicted_output, axis=0)

y_pred = predicted_output.reshape(-1, n_assets)
y_true = y_test.reshape(-1, n_assets)

In [ ]:
# Invert scaling
def invert_standardization(data, M2, factor):
  
    # Consider just volatility series
    M2 = M2[1::2]

    y_pred = factor * y_pred * M2

    return y_pred

# Apply inversion
y_pred = invert_standardization(y_pred, M2, factor)
y_true = invert_standardization(y_true, M2, factor)

In [ ]:
def evaluate(y_true, y_pred, folder):
    """
    Function to calculate MSE and QLIKE
    """

    mse = []
    qlike = []

    for i in range(0, 29):
        mse_i = (y_true[:, i] - y_pred[:, i]) ** 2
        qlike_i = np.log(y_pred[:, i]) + (y_true[:, i] /  y_pred[:, i])

        # save results (point by point)
        results = np.array([mse_i, qlike_i]).transpose()
        np.savetxt(folder + assets[i] + '.csv', results, delimiter=',', header='MSE, Q-LIKE', fmt='%10.5f', comments='')

        mse.append(np.mean(mse_i, axis=0))
        qlike.append(np.mean(qlike_i, axis=0))

    return mse, qlike

In [ ]:
# Apply EVALUATE function to predictions
mse, qlike = evaluate(y_true, y_pred, '1-BASE/')

# save results
results = np.array([mse, qlike]).transpose()
np.savetxt('results/1.csv', results, delimiter=',', header='MSE,Q-LIKE', fmt='%10.5f', comments='')

df = pd.DataFrame({'MSE': mse, 'QLIKE': qlike})
print(df.describe())